This code is meant to test the performance of random dictionary embeddings for classification purposes

In [1]:
import sys
sys.path.append('../')

In [2]:
import torchvision.datasets as datasets
from torch.utils.data import ConcatDataset
from sklearn.model_selection import KFold
import torchvision.transforms as transforms
import numpy as np
import torch
import pandas as pd
import numpy as np
import torch
from botorch.acquisition.analytic import (LogExpectedImprovement, UpperConfidenceBound,
                                        ProbabilityOfImprovement)
from botorch.acquisition import qLogExpectedImprovement, qUpperConfidenceBound, qProbabilityOfImprovement
from botorch.optim import optimize_acqf
from botorch.models import SingleTaskGP
from bodi import HammingEmbeddingDictionary

In [3]:
num_samples = 15
cat_classes = [2, 3, 4, 5, 3, 6, 2, 4, 3, 5]
num_cont = 10
num_cat_features = len(cat_classes)

cat_features = []

for num_classes in cat_classes:
    probs = torch.rand(num_samples, num_classes)
    probs = torch.softmax(probs, dim=1)
    samples = torch.multinomial(probs, num_samples=1).squeeze(1)
    cat_features.append(samples)

cat_var = torch.stack(cat_features, dim=1)
cont_features = torch.rand(num_samples, num_cont)
train_X = torch.cat((cat_var.float(), cont_features), axis=1)

In [4]:
inp = HammingEmbeddingDictionary(cat_dims=[0,1,2,3,4,5,6,7,8,9], 
                                      reduced_cat_dim=2, 
                                      classes_per_cat=cat_classes
                                      #cont bounds
                                      )

train_Y = torch.sin(train_X).sum(dim=1, keepdim=True)
gp = SingleTaskGP(train_X, train_Y, input_transform=inp)

bounds = (torch.min(train_X, 0)[0], torch.max(train_X, 0)[0])
bounds = torch.stack(bounds)

acq_func = qLogExpectedImprovement(gp, train_Y.max())
candidate, acq_value = optimize_acqf(acq_func, bounds=bounds, q=2, num_restarts=5, raw_samples=20,)

C:\Users\danin\AppData\Local\Temp\ipykernel_22464\3225654135.py:8: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  gp = SingleTaskGP(train_X, train_Y, input_transform=inp)
c:\Users\danin\AppData\Local\Programs\Python\Python313\Lib\site-packages\botorch\optim\optimize.py:652: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL: .')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)


In [5]:
from gpytorch.kernels.matern_kernel import MaternKernel
from gpytorch.kernels import ScaleKernel

In [8]:
# Common transform to convert PIL image to tensor
transform = transforms.ToTensor()

# Fashion MNIST
full_fashion = ConcatDataset([
    datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform),
    datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
])

# Digit MNIST
full_mnist = ConcatDataset([
    datasets.MNIST(root='./data', train=True, download=True, transform=transform),
    datasets.MNIST(root='./data', train=False, download=True, transform=transform)
])

def dataset_to_numpy(dataset):
    images = []
    labels = []
    for img, label in dataset:
        images.append(img.numpy())  # shape: (1, 28, 28)
        labels.append(label)
    return np.array(images), np.array(labels)

x_mnist, y_mnist = dataset_to_numpy(full_mnist)
x_fashion, y_fashion = dataset_to_numpy(full_fashion)

x_mnist = x_mnist.reshape((x_mnist.shape[0], -1))
x_fashion = x_fashion.reshape((x_fashion.shape[0], -1))